In [36]:
import pandas as pd

In [37]:
csv_path = '../data/raw/meta/Emergency IKEM 2017-2023.csv' 
df = pd.read_csv(csv_path, delimiter=';')

In [38]:
df.head()

,patient_ID,visit_ID,EntryDate,EntryTime,EndDate,EndTime,age,AdmissionType,AdmissionType_,AdmissionRZP,AdmissionRZP_,VisitReason,VisitReason_,VisitType,VisitType_,DgCode,DischargeTo_Agg,DischargeTo_unit_agg,file
0,1037,1,01.01.2017,10:47:00,01.01.2017,13:14:30,73.9,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.3: Typický flutter síní,outpatient,NaN,MUSE_20231118_022101_39000.xml
1,1037,1,01.01.2017,10:47:00,01.01.2017,13:14:30,73.9,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.3: Typický flutter síní,outpatient,NaN,MUSE_20231118_022102_42000.xml
2,840,2,01.01.2017,11:13:00,01.01.2017,13:34:00,66.5,100,bez doporučení,NaN,? typ záchranky,1,bolest na hrudi,2,opakovaná návštěva,"I47.1: Perzistentní síňová tachykardie, EKV na...",outpatient,NaN,MUSE_20231118_022111_51000.xml
3,840,2,01.01.2017,11:13:00,01.01.2017,13:34:00,66.5,100,bez doporučení,NaN,? typ záchranky,1,bolest na hrudi,2,opakovaná návštěva,"I47.1: Perzistentní síňová tachykardie, EKV na...",outpatient,NaN,MUSE_20231118_022110_48000.xml
4,1500,3,01.01.2017,11:27:00,01.01.2017,14:25:00,78.5,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.0: Paroxyzmální fibrilace síní,outpatient,NaN,MUSE_20231118_022120_59000.xml


## Preprocessing ##

In [39]:
# Combine EntryDate and EntryTime to one datetime column
df['datetime'] = pd.to_datetime(df['EntryDate'] + ' ' + df['EntryTime'], format='%d.%m.%Y %H:%M:%S')

# Drop the original date and time columns
df = df.drop(['EntryDate', 'EntryTime'], axis=1)

# Specify the order of columns for sorting
sort_order = ['patient_ID', 'visit_ID', 'datetime']

# Sort the DataFrame based on the specified columns
df = df.sort_values(by=sort_order)

# Ignore repeated measurements taken during the same visit
df = df.drop_duplicates(subset=['patient_ID', 'visit_ID'], keep='first')

# Filter patients who passed away
df = df[df['DischargeTo_Agg'] != 'exitus']

In [40]:
df.head()

,patient_ID,visit_ID,EndDate,EndTime,age,AdmissionType,AdmissionType_,AdmissionRZP,AdmissionRZP_,VisitReason,VisitReason_,VisitType,VisitType_,DgCode,DischargeTo_Agg,DischargeTo_unit_agg,file,datetime
2230,1,1459,10.04.2017,18:41:19,68.1,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I48.3: Flutter síní paroxysmální při antiarytm...,outpatient,NaN,MUSE_20231122_211154_57000.xml,2017-04-10 16:26:00
3466,1,2253,03.06.2017,19:30:00,68.2,100,bez doporučení,NaN,? typ záchranky,3,palpitace,2,opakovaná návštěva,I47.1: Síňová makroreentry tachykardie s nepra...,outpatient,NaN,MUSE_20231122_221512_86000.xml,2017-06-03 16:08:00
3959,1,2584,28.06.2017,10:38:00,68.3,101,přes RZP/LSPP,P,Praha,3,palpitace,2,opakovaná návštěva,"I48.4: Parox. makroreentry síň. tachykardie, n...",outpatient,NaN,MUSE_20231122_222724_38000.xml,2017-06-28 06:34:00
9887,1,6589,02.05.2018,17:05:00,69.1,101,přes RZP/LSPP,P,Praha,2,dušnost,2,opakovaná návštěva,I47.1: Supraventrikulární tachykardie,cardiology clinic,other,MUSE_20231122_101126_09000.xml,2018-05-02 15:12:00
19395,1,13267,12.02.2020,13:22:00,70.9,101,přes RZP/LSPP,P,Praha,1,bolest na hrudi,2,opakovaná návštěva,I48.1: Přetrvávající (perzistentní) fibrilace ...,outpatient,NaN,MUSE_20231120_222227_88000.xml,2020-02-12 09:56:00


In [41]:
df.to_csv('../data/processed/meta/IKEM_processed.csv', index=False, encoding='utf-8-sig')

In [42]:
df['DischargeTo_Agg'].unique()

array(['outpatient', 'cardiology clinic', 'other'], dtype=object)

In [43]:
len(df)

23292